In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
import os
import numpy as np
from collections import Counter
import re
from datetime import datetime, timedelta

# Kết nối MongoDB
mongo_uri = os.environ.get('MONGO_URI')
client = MongoClient(mongo_uri)
db = client.get_database()
collection = db['pred_news']

# Lấy dữ liệu từ collection
data = list(collection.find())
df = pd.DataFrame(data)

# Thêm cột thời gian và chuyển đổi sang datetime
if 'published_date' in df.columns:
    df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')
    # Sắp xếp theo thời gian
    df = df.sort_values('published_date')
else:
    print("Không tìm thấy cột 'published_date', sử dụng chỉ số để thay thế")
    df['published_date'] = pd.to_datetime('today') - pd.to_timedelta(np.arange(len(df)), 'D')

# Tạo hàm phân tích xu hướng
def analyze_trend(dataframe):
    # Phân tích xu hướng gần đây (30 ngày gần nhất hoặc tất cả nếu ít hơn)
    recent_days = 30
    today = pd.to_datetime('today')
    if 'published_date' in dataframe.columns:
        recent_df = dataframe[dataframe['published_date'] >= (today - timedelta(days=recent_days))]
        if len(recent_df) < 5:  # Nếu không đủ dữ liệu, lấy 30% dữ liệu gần nhất
            recent_df = dataframe.iloc[-int(len(dataframe)*0.3):]
    else:
        recent_df = dataframe.iloc[-int(len(dataframe)*0.3):]
    
    # Phân tích cảm xúc gần đây
    sentiment_counts = recent_df['sentiment'].value_counts()
    sentiment_percentage = (sentiment_counts / sentiment_counts.sum() * 100).round(1)
    
    # Phân tích từ khóa phổ biến
    if 'title' in recent_df.columns:
        titles = ' '.join(recent_df['title'].astype(str).fillna(''))
        words = re.findall(r'\b[A-Za-zÀ-ỹ][A-Za-zÀ-ỹ]+\b', titles)
        
        # Loại bỏ các từ dừng trong tiếng Việt
        vietnamese_stopwords = [
            'và', 'của', 'có', 'cho', 'các', 'với', 'là', 'được', 'trong', 'đã', 'tại', 
            'từ', 'theo', 'những', 'để', 'không', 'này', 'đến', 'về', 'có thể', 'khi',
            'sẽ', 'đang', 'nhiều', 'như', 'năm', 'trên', 'nhưng', 'sau', 'phải', 'cũng',
            'một', 'đây', 'làm', 'hai', 'còn'
        ]
        
        filtered_words = [word.lower() for word in words if word.lower() not in vietnamese_stopwords and len(word) > 2]
        word_counts = Counter(filtered_words)
        
        # Lấy 10 từ phổ biến nhất
        common_words = word_counts.most_common(10)
    else:
        common_words = []
    
    # Phân tích xu hướng cảm xúc theo thời gian
    if len(recent_df) > 5 and 'published_date' in recent_df.columns:
        # Nhóm theo ngày và tính tỷ lệ cảm xúc
        recent_df['date'] = recent_df['published_date'].dt.date
        sentiment_by_date = recent_df.groupby('date')['sentiment'].value_counts(normalize=True).unstack().fillna(0)
        
        if not sentiment_by_date.empty:
            # Tính xu hướng (tăng/giảm) cho mỗi loại cảm xúc
            trend = {}
            for col in sentiment_by_date.columns:
                if len(sentiment_by_date[col]) >= 3:  # Cần ít nhất 3 điểm để xác định xu hướng
                    values = sentiment_by_date[col].values
                    if np.mean(values[-3:]) > np.mean(values[:3]):
                        trend[col] = "tăng"
                    elif np.mean(values[-3:]) < np.mean(values[:3]):
                        trend[col] = "giảm"
                    else:
                        trend[col] = "ổn định"
                else:
                    trend[col] = "không đủ dữ liệu"
        else:
            trend = {"positive": "không đủ dữ liệu", "negative": "không đủ dữ liệu", "neutral": "không đủ dữ liệu"}
    else:
        trend = {"positive": "không đủ dữ liệu", "negative": "không đủ dữ liệu", "neutral": "không đủ dữ liệu"}
    
    return {
        "sentiment_percentage": sentiment_percentage.to_dict(),
        "common_words": common_words,
        "sentiment_trend": trend
    }

# Phân tích dữ liệu
analysis_result = analyze_trend(df)

# Tạo danh sách từ khóa liên quan đến các cổ phiếu/ngành cụ thể
stock_keywords = {
    "ngân hàng": ["ngân hàng", "vietcombank", "techcombank", "sacombank", "vietinbank", "vcb", "tcb", "stb"],
    "bất động sản": ["bất động sản", "đất", "nhà", "căn hộ", "vinhomes", "novaland", "đầu tư", "xây dựng"],
    "chứng khoán": ["chứng khoán", "cổ phiếu", "vn-index", "hnx", "upcom", "ssi", "vnindex"],
    "năng lượng": ["điện", "dầu khí", "năng lượng", "điện lực", "pvn", "pvgas", "nhiệt điện"],
    "công nghệ": ["fpt", "công nghệ", "phần mềm", "số hóa", "vnpay", "viễn thông"],
    "thực phẩm": ["masan", "vinamilk", "sabeco", "thực phẩm", "đồ uống"],
    "hàng không": ["vietjet", "vietnam airlines", "hàng không", "sân bay", "bamboo", "vjc", "hvn", "adb"]
}

# Hàm đưa ra phân tích và lời khuyên đầu tư
def generate_investment_advice(analysis_result, df):
    sentiment_percentage = analysis_result["sentiment_percentage"]
    common_words = analysis_result["common_words"]
    sentiment_trend = analysis_result["sentiment_trend"]
    
    # Tìm từ khóa ngành xuất hiện trong danh sách từ khóa phổ biến
    mentioned_sectors = {}
    for sector, keywords in stock_keywords.items():
        mentions = 0
        for word, count in common_words:
            if any(keyword in word.lower() for keyword in keywords):
                mentions += count
        if mentions > 0:
            mentioned_sectors[sector] = mentions
    
    # Tìm các bài báo với cảm xúc tích cực/tiêu cực cho từng ngành
    sector_sentiment = {}
    for sector, keywords in stock_keywords.items():
        sector_df = df[df['title'].astype(str).str.lower().apply(
            lambda x: any(keyword in x.lower() for keyword in keywords))]
        
        if len(sector_df) > 0:
            pos_pct = len(sector_df[sector_df['sentiment'] == 'positive']) / len(sector_df) * 100
            neg_pct = len(sector_df[sector_df['sentiment'] == 'negative']) / len(sector_df) * 100
            neu_pct = len(sector_df[sector_df['sentiment'] == 'neutral']) / len(sector_df) * 100
            
            sector_sentiment[sector] = {
                'positive': pos_pct, 
                'negative': neg_pct, 
                'neutral': neu_pct,
                'count': len(sector_df)
            }
    
    # Tạo báo cáo phân tích
    advice_text = "# Báo cáo phân tích thị trường chứng khoán dựa trên tin tức\n\n"
    
    # Phân tích tổng quan
    advice_text += "## Tổng quan thị trường\n\n"
    
    # Xác định tâm lý thị trường
    if 'positive' in sentiment_percentage and 'negative' in sentiment_percentage:
        total_sentiment_score = sentiment_percentage.get('positive', 0) - sentiment_percentage.get('negative', 0)
        if total_sentiment_score > 20:
            market_mood = "Tâm lý thị trường đang rất tích cực"
        elif total_sentiment_score > 10:
            market_mood = "Tâm lý thị trường tích cực"
        elif total_sentiment_score < -20:
            market_mood = "Tâm lý thị trường đang rất tiêu cực"
        elif total_sentiment_score < -10:
            market_mood = "Tâm lý thị trường tiêu cực"
        else:
            market_mood = "Tâm lý thị trường trung lập"
    else:
        market_mood = "Không đủ dữ liệu để đánh giá tâm lý thị trường"
    
    advice_text += f"- {market_mood}\n"
    advice_text += f"- Tin tức tích cực: {sentiment_percentage.get('positive', 0):.1f}%\n"
    advice_text += f"- Tin tức tiêu cực: {sentiment_percentage.get('negative', 0):.1f}%\n"
    advice_text += f"- Tin tức trung lập: {sentiment_percentage.get('neutral', 0):.1f}%\n\n"
    
    # Phân tích xu hướng
    advice_text += "## Xu hướng thị trường\n\n"
    
    if 'positive' in sentiment_trend:
        if sentiment_trend['positive'] == "tăng":
            advice_text += "- Tin tức tích cực đang có xu hướng tăng 📈\n"
        elif sentiment_trend['positive'] == "giảm":
            advice_text += "- Tin tức tích cực đang có xu hướng giảm 📉\n"
        else:
            advice_text += "- Tin tức tích cực đang ổn định\n"
    
    if 'negative' in sentiment_trend:
        if sentiment_trend['negative'] == "tăng":
            advice_text += "- Tin tức tiêu cực đang có xu hướng tăng 📈\n"
        elif sentiment_trend['negative'] == "giảm":
            advice_text += "- Tin tức tiêu cực đang có xu hướng giảm 📉\n"
        else:
            advice_text += "- Tin tức tiêu cực đang ổn định\n"
    
    advice_text += "\n## Từ khóa nổi bật gần đây\n\n"
    for word, count in common_words[:7]:
        advice_text += f"- {word}: {count} lần xuất hiện\n"
    
    # Phân tích theo ngành
    advice_text += "\n## Phân tích theo ngành\n\n"
    
    if sector_sentiment:
        # Sắp xếp ngành theo số lần xuất hiện
        sorted_sectors = sorted(sector_sentiment.items(), 
                               key=lambda x: (x[1]['positive'] - x[1]['negative'], x[1]['count']), 
                               reverse=True)
        
        for sector, stats in sorted_sectors:
            advice_text += f"### {sector.title()}\n"
            advice_text += f"- Số tin tức: {stats['count']}\n"
            advice_text += f"- Tích cực: {stats['positive']:.1f}%\n"
            advice_text += f"- Tiêu cực: {stats['negative']:.1f}%\n"
            advice_text += f"- Trung lập: {stats['neutral']:.1f}%\n"
            
            # Đưa ra lời khuyên
            sentiment_balance = stats['positive'] - stats['negative']
            if sentiment_balance > 20 and stats['count'] >= 5:
                advice_text += f"- **Khuyến nghị:** Cân nhắc MUA/NẮM GIỮ cổ phiếu ngành {sector} dựa trên tin tức tích cực\n"
            elif sentiment_balance < -20 and stats['count'] >= 5:
                advice_text += f"- **Khuyến nghị:** Cân nhắc BÁN/TRÁNH cổ phiếu ngành {sector} do tin tức tiêu cực\n"
            elif stats['count'] >= 5:
                advice_text += f"- **Khuyến nghị:** THEO DÕI ngành {sector}, chưa có xu hướng rõ ràng\n"
            else:
                advice_text += f"- **Khuyến nghị:** Cần thêm dữ liệu để đưa ra nhận định chính xác\n"
            
            advice_text += "\n"
    else:
        advice_text += "Không đủ dữ liệu để phân tích theo ngành\n\n"
    
    # Kết luận
    advice_text += "## Kết luận và lời khuyên tổng thể\n\n"
    
    if 'positive' in sentiment_percentage and 'negative' in sentiment_percentage:
        total_sentiment_score = sentiment_percentage.get('positive', 0) - sentiment_percentage.get('negative', 0)
        
        if total_sentiment_score > 30:
            advice_text += "Thị trường chứng khoán đang trong trạng thái RẤT TÍCH CỰC. Đây có thể là thời điểm tốt để xem xét MUA VÀO, đặc biệt là các cổ phiếu trong các ngành được đề cập tích cực ở trên.\n\n"
        elif total_sentiment_score > 15:
            advice_text += "Thị trường chứng khoán đang trong trạng thái TÍCH CỰC. Có thể cân nhắc GIA TĂNG vị thế vào các mã có triển vọng tốt.\n\n"
        elif total_sentiment_score < -30:
            advice_text += "Thị trường chứng khoán đang trong trạng thái RẤT TIÊU CỰC. Nên cân nhắc BẢO TOÀN VỐN, tránh mua vào trong giai đoạn này và có thể xem xét BÁN bớt các cổ phiếu rủi ro cao.\n\n"
        elif total_sentiment_score < -15:
            advice_text += "Thị trường chứng khoán đang trong trạng thái TIÊU CỰC. Nên THẬN TRỌNG và có chiến lược bảo vệ danh mục đầu tư.\n\n"
        else:
            advice_text += "Thị trường chứng khoán đang trong trạng thái TRUNG LẬP. Nên THEO DÕI kỹ thị trường và chỉ giao dịch khi có tín hiệu rõ ràng.\n\n"
    else:
        advice_text += "Không đủ dữ liệu để đưa ra lời khuyên tổng thể.\n\n"
    
    advice_text += "**Lưu ý:** Đây chỉ là phân tích dựa trên tin tức, nhà đầu tư nên kết hợp với phân tích kỹ thuật và cơ bản khác trước khi đưa ra quyết định đầu tư.\n"
    
    return advice_text

# Tạo lời khuyên đầu tư
investment_advice = generate_investment_advice(analysis_result, df)

# Hiển thị lời khuyên
print(investment_advice)

# Lưu báo cáo ra file
with open('bao_cao_dau_tu.md', 'w', encoding='utf-8') as f:
    f.write(investment_advice)

print("\nĐã lưu báo cáo đầu tư vào file 'bao_cao_dau_tu.md'")

# Tạo biểu đồ phân tích tâm lý thị trường theo ngành
if 'sector_sentiment' in locals() and len(sector_sentiment) > 0:
    sectors = list(sector_sentiment.keys())
    positive_values = [sector_sentiment[s]['positive'] for s in sectors]
    negative_values = [sector_sentiment[s]['negative'] for s in sectors]
    
    plt.figure(figsize=(12, 8))
    x = np.arange(len(sectors))
    width = 0.35
    
    plt.bar(x - width/2, positive_values, width, label='Tích cực', color='green')
    plt.bar(x + width/2, negative_values, width, label='Tiêu cực', color='red')
    
    plt.xlabel('Ngành')
    plt.ylabel('Tỷ lệ (%)')
    plt.title('Tâm lý thị trường theo ngành')
    plt.xticks(x, [sector.title() for sector in sectors], rotation=45, ha='right')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('phan_tich_nganh.png', dpi=300)
    plt.show()

/tmp/ipykernel_553/701452365.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Không tìm thấy cột 'published_date', sử dụng chỉ số để thay thế
# Báo cáo phân tích thị trường chứng khoán dựa trên tin tức

## Tổng quan thị trường

- Tâm lý thị trường đang rất tích cực
- Tin tức tích cực: 53.3%
- Tin tức tiêu cực: 30.0%
- Tin tức trung lập: 16.7%

## Xu hướng thị trường

- Tin tức tích cực đang ổn định
- Tin tức tiêu cực đang ổn định

## Từ khóa nổi bật gần đây

- phiếu: 17 lần xuất hiện
- tăng: 11 lần xuất hiện
- chứng: 9 lần xuất hiện
- khoán: 9 lần xuất hiện
- trần: 8 lần xuất hiện
- kịch: 4 lần xuất hiện
- khối: 4 lần xuất hiện

## Phân tích theo ngành

### Năng Lượng
- Số tin tức: 2
- Tích cực: 100.0%
- Tiêu cực: 0.0%
- Trung lập: 0.0%
- **Khuyến nghị:** Cần thêm dữ liệu để đưa ra nhận định chính xác

### Hàng Không
- Số tin tức: 2
- Tích cực: 100.0%
- Tiêu cực: 0.0%
- Trung lập: 0.0%
- **Khuyến nghị:** Cần thêm dữ liệu để đưa ra nhận định chính xác

### Công Nghệ
- Số tin tức: 6
- Tích cực: 66.7%
- Tiêu cực: 0.0%
- Trung lập: 33.3%
- **Khuyến nghị:** Cân nhắc 

/tmp/ipykernel_553/701452365.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_df['date'] = recent_df['published_date'].dt.date
